# Altklausur

In [ ]:
%CREATE pg://localhost
%GUESS_PARSER

In [ ]:
CREATE TABLE Band (
    BId      INTEGER PRIMARY KEY,
    Name     VARCHAR,
    Genre    VARCHAR,
    Gründung VARCHAR,
    Herkunft VARCHAR
);

INSERT INTO Band VALUES
    (1, 'The White Stripes', 'Garage Rock',   1997, 'USA'),
    (2, 'The Raconteurs',    'Bluesrock',     2005, 'USA'),
    (3, 'Crystal Castles',   'Electroclash',  2006, 'Kanada'),
    (4, 'Knife Party',       'Electro House', 2011, 'Australien'),
    (5, 'Big Deal',          'Pop Rock',      2011, 'USA');

In [ ]:
CREATE TABLE Künstler (
    KId         INTEGER PRIMARY KEY,
    Name        VARCHAR,
    Geburtsjahr INTEGER
);

INSERT INTO Künstler VALUES
    (1,  'Jack White',       1975),
    (2,  'Meg White',        1974),
    (3,  'Brendan Benson',   1970),
    (4,  'Jack Lawrence',    1976),
    (5,  'Patrick Keeler',   1975),
    (6,  'Ethan Kath',       1977),
    (7,  'Alice Glass',      1988),
    (8,  'Edith Frances',    NULL),
    (9,  'Rob Swire',        1982),
    (10, 'Gareth McGrillen', 1982),
    (11, 'Mike Kerr',        NULL);

In [ ]:
CREATE TABLE Mitglied (
    KId        INTEGER,
    BId        INTEGER,
    Instrument VARCHAR,
    PRIMARY KEY (KId, BId),
    FOREIGN KEY (KId) REFERENCES Künstler(KId) ON DELETE CASCADE,
    FOREIGN KEY (BId) REFERENCES Band(BId) ON DELETE CASCADE
);

INSERT INTO Mitglied VALUES
    (1,  1, 'Gitarre'),
    (1,  2, 'Schlagzeug'),
    (2,  1, 'Gitarre'),
    (3,  2, 'Gitarre'),
    (4,  2, 'Bass'),
    (5,  2, 'Schlagzeug'),
    (6,  3, NULL),
    (7,  3, NULL),
    (8,  3, NULL),
    (9,  4, 'DJ'),
    (10, 4, 'DJ');

## Aufgabe 3

### a

In [ ]:
SELECT DISTINCT Band.*
FROM Band
JOIN Mitglied USING (BId)
JOIN Künstler USING (KId)
WHERE Geburtsjahr IS NULL

### b

In [ ]:
SELECT BId
FROM (
    SELECT BId, COUNT(*) AS count
    FROM Mitglied
    GROUP BY BId
)
ORDER BY count DESC
LIMIT 1

### c

In [ ]:
SELECT DISTINCT Künstler.*
FROM Künstler
JOIN Mitglied M1 USING (KId)
JOIN Mitglied M2 USING (KId)
WHERE M1.BId != M2.BId

### d

In [ ]:
SELECT Künstler.*
FROM Künstler
LEFT JOIN Mitglied USING (KId)
WHERE Mitglied.KId IS NULL

### e
s.o.

## Aufgabe 4

### a

In [ ]:
π [Name, Geburtsjahr] (Künstler ⋈ Mitglied ⋈ π [bid] (σ [Herkunft = 'USA'] (Band)))

### b

In [ ]:
π [Name] (Band ⋈ (π [BId] (Band) \ π [BId] (Mitglied)))

In [ ]:
π [BId] (
    π [BId, Name] (Band) \ π [BId, Name] (Mitglied ⋈ Band)
)

### c

In [ ]:
{ Instrument | Mitglied(_, BId, Instrument) ∧ Band(BId, _, 'Garage Rock', _, _) }

### d

In [ ]:
{ Gründung, Herkunft | Band(BId, _, _, Gründung, Herkunft) ∧ Mitglied(BId, KId, _) ∧ Künstler(KId, _, Geburtsjahr) ∧ 1975 ≤ Geburtsjahr ∧ Geburtsjahr ≤ 1977 }

In [ ]:
{ Gründung, Herkunft | Band(BId, _, _, Gründung, Herkunft) ∧ Mitglied(BId, KId, _) ∧ Künstler(KId, _, Geburtsjahr) ∧ (Geburtsjahr = 1975 ∨ Geburtsjahr = 1976 ∨ Geburtsjahr = 1977) }

## Aufgabe 5

### d
Hier Formulierung mit PostgreSQL:

In [ ]:
CREATE FUNCTION aufgabefuenfd()
    RETURNS trigger AS
$func$
BEGIN
    DELETE FROM Künstler
    WHERE KId IN (
        SELECT KId
        FROM Mitglied
        WHERE BId = OLD.BId
    );
    RETURN OLD;
END
$func$ LANGUAGE plpgsql;

In [ ]:
CREATE TRIGGER aufgabefuenfd
BEFORE DELETE ON Band
FOR EACH ROW
EXECUTE PROCEDURE aufgabefuenfd()

In [ ]:
SELECT * FROM Künstler

In [ ]:
DELETE FROM Band WHERE BId = 2

In [ ]:
SELECT * FROM Künstler